## Generate Data

In [1]:
# dependences for data loading and sifting
import os
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [2]:
# file paths to data directories
train_dir = "data/train/"
test_dir = "data/test/"
val_dir = "data/val/"

In [3]:
# instatiating a data degenerater for each split sample 
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)
                                   
val_datagen = ImageDataGenerator(rescale=1./255)

train_data_generator = train_datagen.flow_from_directory(
                       train_dir,
                       target_size=(150,150),
                       batch_size=16,
                       class_mode='binary',
                       color_mode='grayscale')

test_data_generator = test_datagen.flow_from_directory(
                      test_dir,
                      target_size=(150,150),
                      batch_size=16,
                      class_mode='binary',
                      color_mode='grayscale')

val_data_generator = val_datagen.flow_from_directory(
                     val_dir,
                     target_size=(150,150),
                     batch_size=16,
                     class_mode='binary',
                     color_mode='grayscale')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


## Baseline Model
Beginning a convolutional neural network with the most simple architecture for the sake of efficient diagnostics and optimization.  

In [6]:
from keras import layers
from keras import models
from keras import optimizers

The architecture begins with a convolutional 2D layers, followed by a pooling layer, flattened, and then followed up with two dense layers.  
THe model is compiled using binary cross entropy to measure the loss function and accuracy foor the performance metric.

In [7]:

# Your code here; design and compile the model
base_model = models.Sequential()
base_model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,1)))
base_model.add(layers.MaxPooling2D((2, 2)))
base_model.add(layers.Flatten())
base_model.add(layers.Dense(512, activation='relu'))
base_model.add(layers.Dense(1, activation='sigmoid'))

base_model.compile(loss='binary_crossentropy',
              metrics=['acc'])

the compiled base model is now fit very lean, using 10 steps per epoch and 5 total epochs, the test data set is used for validation as the actual validation sample is extremely small and reserved only for the final phase of performance evaluation. 

In [12]:
base_model_results = base_model.fit(train_data_generator, 
                              steps_per_epoch=10, 
                              epochs=5, 
                              validation_data=test_data_generator, 
                              validation_steps=15)

Epoch 1/5
10/10 [==============================] - 24s 2s/step - loss: 0.4038 - acc: 0.8562 - val_loss: 0.6068 - val_acc: 0.7167
Epoch 2/5
10/10 [==============================] - 24s 2s/step - loss: 0.2617 - acc: 0.9187 - val_loss: 0.5568 - val_acc: 0.7750
Epoch 3/5
10/10 [==============================] - 24s 2s/step - loss: 0.2517 - acc: 0.9062 - val_loss: 0.3020 - val_acc: 0.8750
Epoch 4/5
10/10 [==============================] - 23s 2s/step - loss: 0.1205 - acc: 0.9500 - val_loss: 0.4314 - val_acc: 0.8083
Epoch 5/5
10/10 [==============================] - 23s 2s/step - loss: 0.2201 - acc: 0.9250 - val_loss: 0.9969 - val_acc: 0.6917


The base model begins stronger than expected with an accuracy of 74% on the test data. A good start, with plenty of room to improve. 